In [1]:
import os

In [2]:
%pwd

'/Users/huseyin/Documents/deep-learning/end-to-end-nlp/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/huseyin/Documents/deep-learning/end-to-end-nlp'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directory

In [7]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size
from tqdm import tqdm


In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            with tqdm(
                unit="B",
                unit_scale=True,
                unit_divisor=1024,
                miniters=1,
                desc="Downloading",
            ) as progress:

                def progress_hook(block_num, block_size, total_size):
                    progress.total = total_size  # Toplam boyut
                    progress.update(block_size)  # İlerleme güncellemesi

                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file,
                    reporthook=progress_hook,  # İlerleme çubuğu için fonksiyon
                )

            logger.info(
                f"{filename} download completed with following info:\n{headers}"
            )
        else:
            file_size = get_size(Path(self.config.local_data_file))
            logger.info(f"File already exists with size: {file_size}")

    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file to the specified directory
        Function return None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-10-25 19:29:33,838: INFO: common Successfully read YAML file from config/config.yaml
[2024-10-25 19:29:33,840: INFO: common Successfully read YAML file from params.yaml
[2024-10-25 19:29:33,841: INFO: common Created directory: artifacts
[2024-10-25 19:29:33,842: INFO: common Created directory: artifacts/data_ingestion
[2024-10-25 19:29:33,842: INFO: 3158102211 File already exists with size: 8076 KB
